# 揭秘链的复杂性

随着链需要承担的责任越来越多，链的内部结构也会变得越来越复杂。这是因为链需要处理更多的任务，需要更多的组件来完成这些任务，而这些组件又可能是其他的链。因此，当我们需要处理更复杂的业务场景时，我们可能需要使用到更复杂的链。

这次教程为说明链之间的关系，流程是首先使用数据连接模块中的LEDVR工作流（这个代码同 ../Data_connection/LEDVR的示例代码.ipynb)，获取到相关文档，然后使用 load_qa_chain, 跟这个文档进行聊天。


In [ ]:
%pip install langchain


In [1]:
# 此为测试密钥和环境的代码。
from langchain.llms import OpenAI
openai_api_key=""

llm = OpenAI(openai_api_key=openai_api_key)
llm.predict("你好")

'\n\n你好！'

每次开始之前，最好用模型包装器测试一下自己的密钥和环境是否配置正确。

首先，我们使用文档加载器L，创建一个WebBaseLoader实例，用于从网络加载数据。在这个例子中，我们加载的是一个博客文章。加载器读取该网址的内容，并将其转换为一份文档数据。

In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("http://developers.mini1.cn/wiki/luawh.html")
data = loader.load()

随后，我们使用文本嵌入模型embedding，将这些分割后的文本数据转换为向量数据。我们创建一个OpenAIEmbeddings实例，用于将文本转换为向量。

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)


In [6]:
len(splits)

79

In [7]:

from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents=splits,embedding=embedding)
vectordb

In [8]:

from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

你可以选择其他加载器。

我们加入记忆模块的记忆，保存对话的历史记录。

In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

实例化 `ConversationalRetrievalChain`

In [11]:

retriever = vectordb.as_retriever()



In [12]:
retriever

VectorStoreRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000018AB4774690>, search_type='similarity', search_kwargs={})

实例化 ConversationalRetrievalChain 链。这个链负责会话和检索相关文档。

In [13]:
llm = OpenAI(openai_api_key=openai_api_key)
qa = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)

我们可以开始提问了。

In [18]:
question = "什么是标识符?"
result = qa({"question": question})
result["answer"]

' 标识符用于定义一个变量，函数或其他用户定义的项。它以一个字母A到Z或a到z或下划线_开头后跟0个或多个字母，下划线，数字（0到9）。Lua不允许使用特殊字符如@，$和%来定义标识符。'

In [23]:
result["answer"]

' 全局变量、局部变量、表中的域。'

In [21]:
query = "Lua 变量有三种类型？"
result = qa({"question": query})

In [22]:
result["answer"]

' 全局变量、局部变量、表中的域。'

## 合并文档的类型设置为 `map_reduce` 类型

ConversationalRetrievalChain 默认的是  `stuff` 类型， 根据不同的需求，可以配置不同的类型，算法不一样，最终的结果也不一样。
 

In [24]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

In [29]:

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

In [26]:
chat_history = []
query = "LUA 是什么？"
result = chain({"question": query, "chat_history": chat_history})

In [27]:
result["answer"]

' Lua is a lightweight, scripting language written in standard C and open in source code form. It is designed to be embedded into applications, providing applications with flexible extension and customization capabilities.'

## 使用QA SOURCE 链条，ConversationalRetrievalChain

我们可以使用load_qa_with_sources_chain，我们可以看到答案的来源位置。

In [30]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [31]:
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

In [34]:
chat_history = []
query = "当变量个数和值的个数不一致时，Lua会一直以变量个数为基础采取什么策略？"
result = chain({"question": query, "chat_history": chat_history})

In [35]:
result["answer"]

' Lua会一直以变量个数为基础，当变量个数多于值的个数时，会按变量个数补足nil，当变量个数少于值的个数时，多余的值会被忽略。\nSOURCES: http://developers.mini1.cn/wiki/luawh.html'

## 使用 `stdout` 的 ConversationalRetrievalChain 



In [36]:
from langchain.chains.llm import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.conversational_retrieval.prompts import (
    CONDENSE_QUESTION_PROMPT,
    QA_PROMPT,
)
from langchain.chains.question_answering import load_qa_chain

# Construct a ConversationalRetrievalChain with a streaming llm for combine docs
# and a separate, non-streaming llm for question generation
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
streaming_llm = OpenAI(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0,
    openai_api_key=openai_api_key,
)

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(streaming_llm, chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator,
)

In [37]:
chat_history = []
query = "Lua的循环语句有哪些？"
result = qa({"question": query, "chat_history": chat_history})

 While循环, for循环, repeat...until, 和循环嵌套.

In [38]:
chat_history = [(query, result["answer"])]
query = "for循环是什么？"
result = qa({"question": query, "chat_history": chat_history})

 for循环是一种数值for循环，它通过指定一个变量从一个值变化到另一个值，每次变化以一个步长递增变量，并执行一次"执行体"。